In [12]:
# ! pip install slack_sdk

In [15]:
import requests, os, re
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import xml.etree.ElementTree as ET
from slack_sdk import WebClient

# Slack setup
SLACK_TOKEN = os.environ["SLACK_TOKEN"]
CHANNEL = "#events-rss"
client = WebClient(token=SLACK_TOKEN)

# Determine current week (Monday–Sunday) in Pacific Time
PACIFIC = ZoneInfo("America/Los_Angeles")
now = datetime.now(PACIFIC)
monday = now.date() - timedelta(days=now.weekday())
sunday = monday + timedelta(days=6)

# RSS feed
URL = "https://calendar.usc.edu/calendar/1.xml"
xml_data = requests.get(URL).content
root = ET.fromstring(xml_data)

# Collect Event Highlights
categories = {}

for item in root.findall(".//item"):
    title = item.findtext("title") or "No title"
    link = item.findtext("link") or ""
    date_str = item.findtext("{http://purl.org/dc/elements/1.1/}date") or item.findtext("pubDate")
    if not date_str:
        continue

    # Parse date
    try:
        if "T" in date_str:
            event_dt = datetime.fromisoformat(date_str)
        else:
            event_dt = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z")
        event_date = event_dt.date()
    except Exception:
        continue

    # Only include events in current week
    if not (monday <= event_date <= sunday):
        continue

    # Filter for Event Highlights only
    cats = [c.text.strip() for c in item.findall("category") if c.text]
    if not cats or "Event Highlights" not in cats:
        continue

    # Clean title and determine primary category
    clean_title = re.sub(r"^[A-Za-z]{3,9}\s+\d{1,2},\s+\d{4}:\s*", "", title)
    primary_cat = next((c for c in cats if c != "Event Highlights"), "General")

    # Avoid duplicates by title
    if clean_title not in {t for e in categories.get(primary_cat, []) for t in e}:
        categories.setdefault(primary_cat, []).append((clean_title, link))

# Compose Slack message
formatted_range = f"{monday.strftime('%b %d')} – {sunday.strftime('%b %d, %Y')}"
border = "──────────────────────────────────────────────"

message_lines = [f"*🌟 Upcoming Event Highlights ({formatted_range})*\n"]

# Sort categories alphabetically for consistency
for cat in sorted(categories.keys()):
    events = categories[cat]
    message_lines.append(f"*{cat}*:")
    for title, link in events:
        message_lines.append(f"• <{link}|{title}>")
    message_lines.append("")

message_lines.append(border)
message = "\n".join(message_lines)

# Post to Slack
resp = client.chat_postMessage(channel=CHANNEL, text=message)
print("✅ Sent highlights" if resp.get("ok") else "❌ Failed:", resp)


✅ Sent highlights {'ok': True, 'channel': 'C09QX1CGREV', 'ts': '1762289030.261629', 'message': {'user': 'U09QJ6KQP98', 'type': 'message', 'ts': '1762289030.261629', 'bot_id': 'B09QX3E10MP', 'app_id': 'A09RCC7JFA4', 'text': "*:star2: Upcoming Event Highlights (Nov 03 – Nov 09, 2025)*\n\n*Arts*:\n• <https://calendar.usc.edu/event/alia-chand-all-that-comes-after|Alia Chand: All That Comes After at Roski Studios Building (IFT)>\n• <https://calendar.usc.edu/event/usc-thornton-studio-guitar-program-showcase-1799|USC Thornton Studio Guitar Program Showcase at Carson Soundstage (CTV 136)>\n• <https://calendar.usc.edu/event/guys-and-dolls|Guys and Dolls at Bing Theatre>\n• <https://calendar.usc.edu/event/usc-thornton-composition-program-recital-4174|USC Thornton Composition Program Recital at Newman Recital Hall (AHF 151)>\n\n*General*:\n• <https://calendar.usc.edu/event/usc-stem-cell-distinguished-speaker-seminar-series|USC Stem Cell Distinguished Speaker Seminar Series at Broad CIRM Center (B